# Create video of the images in the folder

### Import libraries

In [1]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from skimage import exposure, measure

### Set paths and read images

In [2]:
# Set paths and video output name
image_folder = 'data/Varrio_Scanner2'
#image_folder = 'data/Hyde_Scanner1/2018'
video_name = 'Video_Varrio2_V3.mp4'
#video_name = 'Video_Hyde1_V3.mp4'

# Number of pictures in second
fps=3

### Get image paths and set video settings

In [3]:
# Get image names
images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]

# Sort image names
images = sorted(images)

# Set video frame size to 2338x1653 (Scale = 0.5). Orig size 4676 × 3306.
# We must use fixed size, because we can not know size of 
# the pics taken in the future and size is used in parameter tuning.
width = int(1485)
height = int(1050)
dim_vid=(width, height*2)
dim_pic=(width, height)

# Set video settings
video = cv2.VideoWriter(filename=video_name, fourcc=cv2.VideoWriter_fourcc(*'XVID'), fps=fps, frameSize=dim_vid)

# Set text font
font = cv2.FONT_HERSHEY_SIMPLEX

### Create only root images

In [4]:
# Detect roots from image
def light_areas_mask(img_color):
    
    # Filter image by pixel values
    mask = cv2.inRange(im, (112, 168,152),(254, 255,254))

    # Remove isolated pixels
    mask = remove_isolated_pixels(mask, min_size=40, max_size=50000)
    
    # "Make areas fatter by 1 pixel (join roots where is holes in edges)
    mask = cv2.dilate(mask, None, iterations=1)
    
    # Return results
    return mask

def change_areas_mask(im1_color, im2_color):
    # Blur images
    im1_color = cv2.bilateralFilter(im1_color, d=3, 
                                    sigmaColor=15, sigmaSpace=10)
    im2_color = cv2.bilateralFilter(im2_color, d=3, 
                                    sigmaColor=15, sigmaSpace=10)
    
    # Calculate difference between images
    diff = cv2.absdiff(im1_color, im2_color)
    
    # Save temp image
    cv2.imwrite('DIFFERENCE.png', diff)
    
    # Filter difference image by pixel values
   # mask = cv2.inRange(diff, (10, 1, 20), (95, 85 ,90))
    mask = cv2.inRange(diff, (10, 7, 17), (165, 170, 155))
    
    # Remove isolated pixels
    mask = remove_isolated_pixels(mask, min_size=15, max_size=5000)
    
    # "Make areas fatter" to join areas
    mask = cv2.dilate(mask, None, iterations=5)
    
    # Remove isolated pixels
    # "Make areas thinner"
    #change = cv2.erode(change, None, iterations=1)
    

    # Return results
    return mask

# Filter noise by the pixel area
def remove_isolated_pixels(thresh, min_size, max_size):
    # Perform a connected component analysis on the thresholded
    # image and remove components that have size less than thershold
    labels = measure.label(thresh, connectivity=2, background=0)
    mask = np.zeros(thresh.shape, dtype="uint8")
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue
        # otherwise, construct the label mask and count the
        # number of pixels 
        labelMask = np.zeros(thresh.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)
        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if numPixels > min_size and numPixels < max_size:
            mask = cv2.add(mask, labelMask)
            
    # Return cleaned mask
    return mask

def make_circles_around(image, mask):
    # find the contours in the mask, then sort them from left to right
    cnts, hierarchy = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, 
                            cv2.CHAIN_APPROX_SIMPLE)
    
    i=0
    # loop over the contours
    for (i, c) in enumerate(cnts):
    # draw the bright spot on the image
        (x, y, w, h) = cv2.boundingRect(c)
        ((cX, cY), radius) = cv2.minEnclosingCircle(c)
        # Tmp radius
        radius=30
        cv2.circle(image, (int(cX), int(cY)), int(radius), (0, 255, 0), 2)
   
    # Return image
    return image, i

### Convert pics to video and save output

In [5]:
# Initialize roots locations binary array
roots=np.zeros((height, width), np.uint8)

# Make video of images. Use tqdm to see the progress.
for i in tqdm(range(len(images))):
    
    # Get present day image name
    image = images[i]
    
    if i==0:
        # Read image
        im = cv2.imread(os.path.join(image_folder, images[i]))
        # Normalize image
        im = cv2.normalize(im, im, 0, 255, cv2.NORM_MINMAX)
        # Resize image
        im = cv2.resize(im, dim_pic,interpolation = cv2.INTER_AREA)
        # Get new root location pixels
        new_roots=light_areas_mask(im)
        # Add new root locations pixels to previous root locations pixels
        roots = new_roots+roots
        
    else:
        # Read images
        im_yesterday = cv2.imread(os.path.join(image_folder, images[i-1]))
        im = cv2.imread(os.path.join(image_folder, images[i]))
        
        # Normalize image
        im_yesterday = cv2.normalize(im_yesterday, im_yesterday, 0, 255, cv2.NORM_MINMAX)
        im = cv2.normalize(im, im, 0, 255, cv2.NORM_MINMAX)
        
        # Resize images
        im_yesterday = cv2.resize(im_yesterday, dim_pic, interpolation = cv2.INTER_AREA)
        im = cv2.resize(im, dim_pic, interpolation = cv2.INTER_AREA)
        
        # Get new root location pixels
        light_mask = light_areas_mask(im)
        change_mask = change_areas_mask(im_yesterday, im)
        new_roots = light_mask & change_mask
        
        # Remove noise from mask
        new_roots = remove_isolated_pixels(new_roots, min_size=40, max_size=5000)

        # Add new root locations pixels to previous root locations pixels
        roots = new_roots+roots
        
    # Mask root locations to image (get colors to root pixels)
    im_masked = cv2.bitwise_or(im, im, mask=roots)
    
    # Make circle around the new root tips
    im, nbr_cnts = make_circles_around(im, new_roots)
    
    # Add number of roots to lower corner
    cv2.putText(img=im_masked, text="Found growth in {} roots".format(nbr_cnts), org=(50,1000), 
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, 
                color=(255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
    
    
    # Concat original image and roots image vertically
    im = np.concatenate((im, im_masked), axis=0)
    
    # Get image date from the image name
    # Date is in different location for Hyde and Varrio
    #date = image[20:30] # For Hyde Scanners
    date = image[14:24] # For Varrio Scanners
    
    # Add date to upper left corner
    cv2.putText(img=im, text=date, org=(30, 90), fontFace=font, 
                fontScale=2, color=(255, 255, 255), thickness=2, 
                lineType=cv2.LINE_AA)
    
    # Add frame to video
    video.write(im)
     
# Close windows
cv2.destroyAllWindows() 

# Save output video
video.release()

100%|██████████| 184/184 [2:40:58<00:00, 52.49s/it]   
